# Etude d'un cas

In [1]:
!pip install equinox

In [2]:
%reset -f

In [3]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import seaborn as sns
import time

import jax
from jax import vmap, jit
import jax.numpy as jnp
import jax.random as jr
import equinox as eqx
from jax import lax

### Importation

Nos données ont été importée depuis l'url

    https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip

Pour être indépendant de toute mise à jour de ce lien, je les ai placé sur mon serveur.

## Le jeu de données météo

Cet séries temporelle contient 14 caractéristiques (features)  telles que la température de l'air, la pression atmosphérique, l'humidité... Celles-ci ont été collectées toutes les 10 minutes, à partir de 2003.


In [4]:
data_frame_10min = pd.read_csv("http://octaviogame.com/liens/data/jena_climate_2009_2016.csv", sep=",")

In [5]:
data_frame_10min

L'indexe n'est pas la colonne "Date Time". Modifions cela:

In [6]:
#Convertir la colonne en datetime
data_frame_10min['Date Time'] = pd.to_datetime(data_frame_10min['Date Time'],format='%d.%m.%Y %H:%M:%S')
data_frame_10min = data_frame_10min.set_index('Date Time')
data_frame_10min

In [7]:
data_frame_10min.describe()

On voit qu'on a la valeur "-9999.000000" qui semble abhérente dans les colonnes "wv".

In [8]:
data_frame_10min_clean=data_frame_10min.replace(-9999.0,0)

In [9]:
data_frame_10min_clean.isna().sum()

On ne garde qu'une mesure par heure:

In [10]:
data_frame=data_frame_10min_clean.resample('h').mean().dropna()
data_frame

In [11]:
data_frame.isna().sum()#date_time = pd.to_datetime(data_frame.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

Voici l'évolution de quelques features au fil du temps.

In [12]:
plot_cols = ['p (mbar)','T (degC)',  'rho (g/m**3)']
plot_features = data_frame[plot_cols]
#plot_features.index = date_time

In [13]:
plot_features.plot(subplots=True,figsize=(10,10));

### Transformons le vent

La dernière colonne des données, wd (deg) , donne la direction du vent en unités de degrés. Les angles ne font pas de bonnes entrées de modèle, 359 ° et 0 ° doivent être proches l'un de l'autre ! La direction ne devrait pas avoir d'importance si le vent ne souffle très peu.

Pour l'instant, la distribution des données éoliennes ressemble à ceci:

In [14]:
plt.hist2d(data_frame['wd (deg)'], data_frame['wv (m/s)'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]');


Mais cela sera plus facile à interpréter pour le modèle si on converti ceci en un "vecteur de vent" qui indique la direction et la puissance du vent.

In [15]:
wv = data_frame['wv (m/s)']
max_wv = data_frame['max. wv (m/s)']

In [16]:
wd_rad = data_frame['wd (deg)']*np.pi / 180

In [17]:
# Calculate the wind x and y components.
data_frame['Wx'] = wv*np.cos(wd_rad)
data_frame['Wy'] = wv*np.sin(wd_rad)

In [18]:
# Calculate the max wind x and y components.
data_frame['max Wx'] = max_wv*np.cos(wd_rad)
data_frame['max Wy'] = max_wv*np.sin(wd_rad)

La distribution des vecteurs de vent est beaucoup plus simple à interpréter pour le modèle (et pour nous aussi).

In [19]:
fig,ax=plt.subplots()
ax.hist2d(data_frame['Wx'], data_frame['Wy'], bins=(50, 50), vmax=400)
ax.set_xlabel('Wind X [m/s]')
ax.set_ylabel('Wind Y [m/s]');

In [20]:
fig,ax=plt.subplots()
ax.hist2d(data_frame['max Wx'], data_frame['max Wy'], bins=(50, 50), vmax=400)
ax.set_xlabel('Wind X [m/s]')
ax.set_ylabel('Wind Y [m/s]');

***A vous:*** Quels sont les vents dominants ?

### Transformons le temps

Je récupère mon temps en seconde.

In [21]:
timestamp_s = data_frame.index.map(datetime.datetime.timestamp)

Tout comme la direction du vent, le temps en secondes n'est pas une entrée facile à interpréter pour les modèles: Rappelons que c'est modèles font des multiplications/addition/activation sur les données de bases et que les paramètres appris le sont sur une base statistique. Or le temps en seconde est une variable qui a une grande amplitude, et de plus chaque valeur apparait une unique fois! Pas facile de faire des stats avec des quantités qui ne se répettent pas.



Or les données météorologiques suivent des périodicités quotidienne et annuelle très naturelles.
Une technique simple consiste alors à utiliser sin et cos pour convertir l'heure absolue en un encodage indiquant la "partie du jour" et "partie de l'année" (c'est les coordonnée des éguilles des horloges):

In [22]:
day = 24*60*60
year = (365.2425)*day


data_frame['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
data_frame['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
data_frame['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
data_frame['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [23]:
plt.plot(np.array(data_frame['Day sin'])[:100])
plt.plot(np.array(data_frame['Day cos'])[:100])
plt.xlabel('Time [h]')
plt.title('Time of day signal');

Cette convertion permet au modèle d'accéder aux caractéristiques fréquentielle les plus importantes. Dans notre cas, nous savions à l'avance quelles fréquences étaient importantes.

Si on ne connaissait pas les fréquences principales, on aurait pu les déterminer avec la `fft`:

In [24]:
fft = tf.signal.rfft(data_frame['T (degC)'])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(data_frame['T (degC)'])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

### On supprime les noms de colonnes

In [25]:
data_frame.columns

In [26]:
data_frame_clean=data_frame.drop(['wv (m/s)', 'max. wv (m/s)','wd (deg)'],axis=1)

Avec `.values` on ne garde que oublie les noms des colonnes.

In [27]:
MAT_np=data_frame_clean.values.astype(np.float32)
MAT_np.shape

In [28]:
MAT_jnp=jnp.array(MAT_np)

In [29]:
MAT_jnp.devices()

⇑ si vous avez accès à un GPU, les données sont maintenant sur GPU (sinon cela fonctionnera plus lentement).

### Séparation Train/Val/Test

On utilise le découpage `(70%, 20%, 10%)` pour les ensembles training, validation, et test.

On découpe par grand segment temporelles:

In [30]:
n = len(data_frame)
n1,n2=int(n*0.7), int(n*0.9)


TRAIN_MAT = MAT_jnp[0:n1]
VAL_MAT = MAT_jnp[n1:n2]
TEST_MAT = MAT_jnp[n2:]

TRAIN_MAT.shape, VAL_MAT.shape, TEST_MAT.shape

### Normalisation


La moyenne et l'écart type doivent être calculés uniquement à l'aide des données d'apprentissage afin que les modèles n'aient pas accès aux valeurs des ensembles de validation et de test.

On peut également dire que le modèle ne devrait pas avoir accès aux valeurs futures de l'ensemble d'entraînement lors de l'entraînement. Ainsi la normalisation devrait être effectuée à l'aide de moyennes mobiles non-anticipative. Mais par souci de simplicité, ce tutoriel utilise une moyenne simple.


In [31]:
train_mean = TRAIN_MAT.mean(axis=0,keepdims=True)
train_std = TRAIN_MAT.std(axis=0,keepdims=True)
train_mean.shape,train_std.shape

In [32]:
TRAIN_MAT = (TRAIN_MAT - train_mean) / train_std
VAL_MAT = (VAL_MAT - train_mean) / train_std
TEST_MAT = (TEST_MAT - train_mean) / train_std

In [33]:
TRAIN_MAT.shape, VAL_MAT.shape, TEST_MAT.shape

### Un plot de vérif

In [34]:
titles=list(data_frame_clean.keys())

In [35]:
fig,axs=plt.subplots(19,1,figsize=(2,20),sharex="all")
for i in range(19):
    axs[i].hist(TRAIN_MAT[:,i],bins=50)
    axs[i].set_title(titles[i])
fig.tight_layout()

***A vous:*** Comment expliquer la forme bizarre des derniers histogrammes ?

## Fenêtrage

On va utiliser la fonction suivante pour fenétrer nos données. Observons là sur des données bidons:

In [36]:
#données bidons pour les tests.
nb_t, dim = 8, 3
T = jnp.arange(0, nb_t)
ones = jnp.ones([ nb_t,dim])
DATA_dummy = ones * T[ :, None]

In [37]:
DATA_dummy

⇑ une série temporelle avec `seq_len`=8 et de dimension 3.

### tous les décalages possibles

In [38]:
def make_consecutive_windows(data,window_size):
    nb_t = data.shape[0]
    nb_possible_windows = nb_t - window_size + 1
    all_shifts = []
    for i in range(window_size):
        # on crée tous les décalages possibles.
        all_shifts.append(data[i:nb_possible_windows + i])

    all_shifts_stack = jnp.stack(all_shifts, axis=0)
    dim=len(all_shifts_stack.shape)
    axes=tuple(range(dim))
    axes_perm=(1,0)+axes[2:]

    return jnp.transpose(all_shifts_stack,axes_perm)

In [39]:
def make_consecutive_windows(data,window_size):
    nb_t = data.shape[0]
    nb_possible_windows = nb_t - window_size + 1
    all_shifts = []
    for i in range(window_size):
        # on crée tous les décalages possibles.
        all_shifts.append(data[i:nb_possible_windows + i])

    all_shifts_stack = jnp.stack(all_shifts, axis=0)

    #les fenêtres sont les colonnes de la matrice. On les transforme en ligne.
    return all_shifts_stack.T

In [40]:
def test():
    res=make_consecutive_windows(DATA_dummy,window_size=4) #b,nb_window*window_size (all shift),window_size,nb_part,dimX
    print(res)
    print(res.shape)
test()

On a cette shape `(5,4,3)` car:

* 5 fenêtres possibles
* chacune de taille 4
* dimensions des données pour un temps: 3

### Par batch

Voici maintenant une classe qui permet de batcher des données

In [ ]:
def batcher(data,batch_size):
    size0 = data.shape[0]

    data=data[np.random.permutation(size0)]
    nb_batch=size0//batch_size


    for i in range(nb_batch):
        batch=data[i*batch_size:(i+1)*batch_size]
        yield batch

In [ ]:
def test():
    data=jnp.arange(22)
    for x in batcher(data,5):
        print(x)
test()

Mais pour notre sujet, les éléments du batchs sont des fenêtres

In [ ]:
def test():
    data_window=make_consecutive_windows(DATA_dummy,window_size=4) #b,nb_window*window_size (all shift),window_size,nb_part,dimX

    for batch in batcher(data_window,batch_size=2):
        print(batch)
        print("-------------")
test()

Remarquons qu'on peut aussi tout distribuer en un seul batch:

In [ ]:
def test():
    data_window=make_consecutive_windows(DATA_dummy,window_size=4) #b,nb_window*window_size (all shift),window_size,nb_part,dimX

    for batch in batcher(data_window,batch_size=len(data_window)):
        print(batch)
        print("-------------")
test()

### L'output

Pour un problème de prédiction, l'output, c'est simplement l'input décaler dans le temps:



* L'input c'est une fenêtre de taille $N$:

$
X_{0} \ X_{1} \ X_{2} \ X_{3} \ \dots \ \dots \ \dots\ \dots X_{N-1}   
$


* L'output c'est cette fenêtre translaté d'un shift $s$ imposé:

$
X_{s} \ X_{s+1} \ X_{s+2} \ X_{s+3} \ \dots X_{s+N-1}   
$





    



On va paramétrer notre distributeur de données avec:
* `input_duration`
* `shift`


In [ ]:
class DataDealer:
    def __init__(self,data,input_duration,shift,batch_size):

        self.dimension_to_keep=2 #température et pression


        if input_duration is None:
            input_duration=data.shape[0]-shift #taille maximale: une seule fenêtre

        self.input_duration=input_duration
        self.output_duration=input_duration

        window_size=input_duration+shift
        assert window_size<=data.shape[0] , f"On a input_duration={input_duration} et shift={shift}. Or la somme des deux doit être inférieur {data.shape[0]} qui est la longueur de la série temporelle "


        self.data_in_windows=make_consecutive_windows(data,window_size)


        if batch_size is not None:
            self.batch_size=batch_size
        else:
            self.batch_size=len(self.data_in_windows)#on passe tout en un seul batch


    def one_epoch_iterator(self):
        for window in batcher(self.data_in_windows,self.batch_size):

            yield window[:,:self.input_duration,:],window[:,-self.output_duration:,:self.dimension_to_keep]



Testons:

In [ ]:
def test():
    data_dealer=DataDealer(DATA_dummy,input_duration=3,shift=2,batch_size=2)

    for x,y in data_dealer.one_epoch_iterator():
        print("x")
        print(x)
        print("y")
        print(y)
        print("-----------")
test()

In [ ]:
def test():
    #un seul batch et une fenêtre de taille maximale
    data_dealer=DataDealer(DATA_dummy,input_duration=None,shift=2,batch_size=None)

    for x,y in data_dealer.one_epoch_iterator():
        print("x")
        print(x)
        print("y")
        print(y)
        print("-----------")
test()

### Illustration graphique sur des données bidons

In [ ]:
def plot_ds(data,input_duration,shift):

    batch_size=2


    data_dealer=DataDealer(data,input_duration,shift,batch_size)


    X,Y = next(data_dealer.one_epoch_iterator())

    fig,axs=plt.subplots(batch_size,2,figsize=(15,10),sharex="all")

    output_duration=input_duration
    deb_output=input_duration+shift-output_duration
    abs_output=np.arange(deb_output,deb_output+output_duration)

    abs_input=np.arange(input_duration)

    for i in range(batch_size):

        axs[i,0].plot(abs_input, X[i,:,0],"o-",label="X")
        axs[i,0].plot(abs_output,Y[i,:,0],"+-",label="Y")

        axs[i,1].plot(abs_input, X[i,:,1],"o-",label="X")
        axs[i,1].plot(abs_output,Y[i,:,1],"+-",label="Y")

    plt.legend()
    axs[0,0].set_title("température")
    axs[0,1].set_title("pression")

In [ ]:
plot_ds(DATA_dummy,input_duration=3,shift=4)

In [ ]:
plot_ds(DATA_dummy,input_duration=3,shift=1)

⇑ Ce n'est pas grave si l'input et l'output se superposent, car les modèles RNN sont non-anticipatif:
$$
\forall t \qquad model(X_t) = fonction(X_t,X_{t-1},X_{t-2},...)
$$

Sur les vraies données

In [ ]:
TRAIN_MAT.shape

In [ ]:
plot_ds(TRAIN_MAT,input_duration=12,shift=2)

## PARAMETRE GLOBAUX

In [ ]:
SHIFT=3 #on prédit 3 heures à l'avance
N_EPOCH= 10 #Cela ira assez vite. N'hésitez pas à augmenter ce chiffre

In [ ]:
import pickle
from dataclasses import dataclass
from typing import Callable
import optax

## Modèle et agent

### Modèle

In [ ]:
class RNN_layer(eqx.Module):
    hidden_size: int
    cell: eqx.Module

    def __init__(self, in_size, out_size, hidden_size,cell_type,rkey):
        assert cell_type in ["gru","lstm"]

        self.hidden_size = hidden_size
        if cell_type=="gru":
            self.cell = eqx.nn.GRUCell(in_size, hidden_size, key=rkey)
        else:
            self.cell = eqx.nn.LSTMCell(in_size, hidden_size, key=rkey)


    def __call__(self, input):

        h_init = jnp.zeros((self.hidden_size,))

        def f(carry, inp):
            h=self.cell(inp, carry)
            #return 2 fois h: le premier pour être utilisé par le prochain appelle de la cellule, le second pour être stocker dans les outputs
            return h, h

        h_final, output = lax.scan(f, h_init, input)

        return output

In [ ]:
class RNN_model(eqx.Module):
    layers: list[eqx.Module]
    final_layer: eqx.nn.Linear
    initial_layer: eqx.nn.Linear

    def __init__(self, in_size, out_size, hidden_size, n_layer,cell_type, rkey):

        self.initial_layer = eqx.nn.Linear(in_size, hidden_size, key=rkey)
        self.final_layer = eqx.nn.Linear(hidden_size, out_size, key=rkey)

        self.layers=[]
        for _ in range(n_layer):
            rk,rkey=jr.split(rkey)
            self.layers.append(RNN_layer(hidden_size, hidden_size, hidden_size,cell_type,rk))


    def __call__(self, input):

        X=vmap(self.initial_layer)(input)
        for layer in self.layers:
            X=layer(X)
        return vmap(self.final_layer)(X)

***A vous:*** Expliquer à quoi sert les 2 vmap qui apparaissent dans la méthode `__call__`

Remarque: on pourrait vouloir mettre ces 2 `vmap` dans le constructeur:


    def __init__(self, in_size, out_size, hidden_size, n_layer,cell_type, rkey):

            self.initial_layer = vmap(eqx.nn.Linear(in_size, hidden_size, key=rkey))
            self.final_layer = vmap(eqx.nn.Linear(hidden_size, out_size, key=rkey))


Mais dans ce cas on a un warning d'equinox très explicite: si l'on applique un transformation-jax dans le constructeur, les paramétres de la fonctions transformée ne seront plus collectée par le `eqx.partition`, et donc ne seront pas entrainée.




In [ ]:
def test():
    seq_len=24
    in_size=7
    out_size=3
    hidden_size=32
    input=jnp.ones([seq_len,in_size])
    n_layer=2
    cell_type="gru"
    rkey=jr.key(0)
    model=RNN_model(in_size, out_size, hidden_size, n_layer,cell_type, rkey)
    assert model(input).shape == (24,3)
test()

In [ ]:
def model_fnm(hyper_param):

    in_size=19
    out_size=2
    hidden_size=hyper_param["hidden_size"]
    n_layer=hyper_param["n_layer"]
    cell_type=hyper_param["cell_type"]


    def model_init(rkey):
        model_eqx=RNN_model(in_size, out_size, hidden_size, n_layer,cell_type, rkey)
        param,_ = eqx.partition(model_eqx,eqx.is_array)
        return param


    model_eqx=RNN_model(in_size, out_size, hidden_size, n_layer,cell_type, jr.key(0))

    def model_apply(param,inp):
        _,static = eqx.partition(model_eqx,eqx.is_array)
        return eqx.combine(static,param)(inp)


    return model_init, jit(model_apply)



def test():#on fait un test en batch
    hyper_param={"hidden_size":32,"n_layer":2,"cell_type":"gru"}
    batch_size=13
    inpV=jnp.ones([batch_size,24,19])

    model_init,U_of_param_inp=model_fnm(hyper_param)

    U_of_param_inpV=vmap(U_of_param_inp,[None,0])
    param=model_init(jr.key(0))
    assert U_of_param_inpV(param,inpV).shape == (batch_size,24,2)
test()


### fonctions de loss et d'update

In [ ]:
def jit_creator(U_of_param_inpV,optimizer):
    @jax.jit
    def loss_compute(params, X,Y):
        Y_pred=U_of_param_inpV(params,X)
        return jnp.mean((Y_pred-Y)**2)

    @jax.jit
    def update_model_param(optimizer_state, model_param, X,Y):
        grads = jax.grad(loss_compute)(model_param, X,Y)
        updates, optimizer_state = optimizer.update(grads, optimizer_state)
        #here the model_param is modified
        model_param = optax.apply_updates(model_param, updates)
        return optimizer_state, model_param

    return loss_compute,update_model_param

### Fonctions de sauvegarde

In [ ]:
def save_as_pickle(file_name,serializable):
    pickle.dump(serializable,open(file_name,"wb"))
def load_from_pickle(file_name):
    return pickle.load(open(file_name,"rb"))
def save_as_str(file_name,serializable):
    with open(file_name, "wt") as f:
        f.write(str(serializable))
def load_from_str(file_name):
    with open(file_name, "rt") as f:
        res = eval(f.read())
    return res

### L'Agent en personne

In [ ]:
import time

@dataclass
class AgentResult:
    hyper_param:dict
    best_loss:float
    score:float
    model_param:dict
    U_of_param_inp:Callable


class Agent:
    @staticmethod
    def load(folder):
        assert os.path.exists(folder),f"folder:{folder} does not exist"
        model_param = load_from_pickle(f"{folder}/model_param")
        best_loss = load_from_str(f"{folder}/best_loss")
        score = load_from_str(f"{folder}/score")
        model_param=load_from_pickle(f"{folder}/model_param")
        hyper_param=load_from_str(f"{folder}/hyper_param")
        _,U_of_param_inp=model_fnm(hyper_param)
        return AgentResult(hyper_param,best_loss,score,model_param,U_of_param_inp)


    @staticmethod
    def train(folder,hyper_param,n_epoch=N_EPOCH,verbose=True):


        train_data_dealer=DataDealer(TRAIN_MAT,input_duration=hyper_param["input_duration"],shift=SHIFT,batch_size=hyper_param["batch_size"])

        #batch_size=None, toutes les données en 1 seul batch
        val_data_dealer=DataDealer(VAL_MAT,input_duration=hyper_param["input_duration"],shift=SHIFT,batch_size=None)
        x_val,y_val=next(val_data_dealer.one_epoch_iterator())

        model_init, U_of_param_inp = model_fnm(hyper_param)

        U_of_param_inpV=vmap(U_of_param_inp,[None,0])

        optimizer = optax.adam(hyper_param["learning_rate"])
        batch_size = hyper_param["batch_size"]

        if os.path.exists(folder):
            if verbose:
                print(f"Existing folder:{folder}, we load model_param and best_loss from if")
            model_param = load_from_pickle(f"{folder}/model_param")
            best_loss =load_from_str(f"{folder}/best_loss")

        else:#c'est la première fois qu'on teste cet hyper_paramètre.
            os.makedirs(folder, exist_ok=True)
            if verbose:
                print(f"New folder:{folder}, model_param are randomly initialized")
            model_param=model_init(jr.key(0))
            best_loss=1e10#l'infini ou presque
            save_as_pickle(f"{folder}/model_param", model_param)
            save_as_str(f"{folder}/best_loss", best_loss)

        save_as_str(f"{folder}/hyper_param", hyper_param)
        optimizer_state=optimizer.init(model_param)

        loss_compute, update_model_param=jit_creator(U_of_param_inpV,optimizer)


        ti0=time.time()
        for _ in range(n_epoch):

            for x,y in train_data_dealer.one_epoch_iterator():
                optimizer_state, model_param = update_model_param(optimizer_state, model_param, x,y)

            val_loss=loss_compute(model_param, x_val, y_val)
            if val_loss <= best_loss:
                best_loss=val_loss
                if verbose:
                    print(f"⬊{val_loss:.3g}", end="")
                save_as_pickle(f"{folder}/model_param",model_param)
                save_as_str(f"{folder}/best_loss",best_loss)
            else:
                if verbose:
                    print(".",end="")
        if verbose:
            print("| end of the optimization loop.")

        val_loss.block_until_ready()
        duration=time.time()-ti0

        score=best_loss*duration
        save_as_str(f"{folder}/score",score)

        return score #utile si on utiliser hyperopt par exemple.


### Baseline

Il est toujours bon d'avoir une prédiction triviale (une BaseLine) que nos modèles plus complexe doivent absolument battre.

Pour prédire la température dans le future, on peut simplement donner la température actuelle. C'est valable si le future n'est pas trop lointain.


Ainsi si notre série temporelle est `data`. L'entrée c'est


    X=data[:-shift,:]

La cible c'est

    Y=data[shift:,[0,1]]

La prédiction triviale c'est simplement:

    Y_pred=X[:-shift,[0,1]]



***A vous:***

* si `shift=1` la baseline sera meilleure que si `shift=12`
* mais si `shift=24` la baseline sera meilleure que si `shift=12`

Pourquoi ?



In [ ]:
def evaluate_base_line(shift):
    X=VAL_MAT
    Y=X[shift:,[0,1]]
    Y_pred=X[:-shift,[0,1]]
    return jnp.mean(jnp.square(Y-Y_pred))

In [ ]:
mse_baseline=evaluate_base_line(SHIFT)
mse_baseline

## Entrainement

###  c'est parti

In [ ]:
import shutil

mother_folder="varying_model_capacity"
#on part de zéro
shutil.rmtree(mother_folder,ignore_errors=True)

for n_layer in [1,3]:
    for hidden_size in [32,128]:
        hyper_param={"batch_size":256,"hidden_size":hidden_size,"n_layer":n_layer,"cell_type":"gru","input_duration":24,"learning_rate":1e-3}
        folder=f"{mother_folder}/n_layer:{n_layer},hidden_size:{hidden_size}"
        score=Agent.train(folder,hyper_param)
        print(f"score:{score}")

⇑ Ensuite c'est à vous de juger si vous voulez favoriser la rapidité ou la précision. En tout cas, tous les modèles font mieux que la base-line (ouf).  

### Observons la prédiction sur le jeu Test



In [ ]:
def plot_prediction(folders,shift=SHIFT,add_baseline=False,up=200,model_names=None):
    #on ne trace que les 'up' premiers temps
    data=TEST_MAT


    #La cyble, c'est les data décalées
    Y=data[shift:,[0,1]] #(nb_t-shift,2)
    fig,axs=plt.subplots(2,1,figsize=(15,6),sharex="all")

    axs[0].plot(Y[:up,0],label="true")
    axs[1].plot(Y[:up,1],label="true")

    axs[0].set_title("pressure")
    axs[1].set_title("temperature")

    if add_baseline:
        #la baseline, c'est de renvoyer simplement les data.
        Y_baseline=data[:-shift,:3]
        axs[0].plot(Y_baseline[:up,0],label="baseline",alpha=0.5)
        axs[1].plot(Y_baseline[:up,1],label="baseline",alpha=0.5)


    x=data[:-shift,:]   #(1,nb_t-shift,19)
    for i,folder in enumerate(folders):
        agentResult=Agent.load(folder)
        U_of_inp=lambda inp : agentResult.U_of_param_inp(agentResult.model_param,inp)

        Y_pred=U_of_inp(x)[:,:3]
        label="pred"
        if model_names is not None:
            label+=model_names[i]
        axs[0].plot(Y_pred[:up,0],label=label)
        axs[1].plot(Y_pred[:up,1],label=label)


    axs[0].legend()

In [ ]:
folder_mini=f"{mother_folder}/n_layer:{1},hidden_size:{32}"
folder_maxi=f"{mother_folder}/n_layer:{3},hidden_size:{128}"

In [ ]:
plot_prediction([folder_mini,folder_maxi],add_baseline=True,model_names=[" model mini"," model maxi"])

### Influence de la longueur des séquences

Pour l'entrainement d'un RNN, les grandes fenêtres sont préférables. Mais il ne faut pas non plus prendre des fenêtre temporelles trop longue car:

* On pourra produire moins de données indépendantes (les longues fenêtre ayant tendance à se superpose)
* et il y a le phénomène de la disparition du gradient: si l'on prend des fenêtres d'entrainement trop longue, le gradient l'erreur a du mal à traverser toutes les cellules RNN: la dérivée d'une longue composition de fonction crée de longs produits qui finissent par être nul (plus d'apprentissage) ou infini (explosion du gradient, échec de l'apprentissage).


Pour l'évaluation d'un RNN, autant prendre la fenêtre la plus longue possible. Cela n'a pas d'inconvénient, et il n'y a qu'un seul échauffement à faire au tout début.

## Feature importance

Comment savoir si une feature a plus d'importance qu'une autre dans le modèle.

Une technique simple est de calculer la dérivée du résultat du modèle par rapport à chacune des features.

Notre modèle est une fonctions comme ceci:
\begin{align}
M :  \mathbb R^{T} *\mathbb R^{19}& \to \mathbb R^{S}*\mathbb R^2 \\
 (x_{t,i})  &\to M(x)
\end{align}
Dans notre exemple $S=T$.

La loss c'est une fonction comme ceci:
\begin{align}
L :  \mathbb R^{T} *\mathbb R^{19} &\to \mathbb R \\
(x_{t,i})  &\mapsto L(x) = \|M(x) - Y_{true} \|^2
\end{align}



On cacul d'abord la dérivée par rapport à chaque entrée de la série temporelle:
$$
G_{t,i} := \partial_{x_{t,i}} L
$$
Puis on somme sur les temps:
$$
G_i =\sum_t |G_{t,i}|
$$

In [ ]:
folder=f"{mother_folder}/n_layer:{3},hidden_size:{128}"
agentResult=Agent.load(folder)
U_of_ϴ_x=agentResult.U_of_param_inp
ϴ=agentResult.model_param

In [ ]:
#les inputs de validation, que l'on renome x par simplicité
x=VAL_MAT[:-SHIFT,:]
x.shape

In [ ]:
y_true=VAL_MAT[SHIFT:,[0,1]]
y_true.shape

In [ ]:
def loss_of_x(x):
    y_pred=U_of_ϴ_x(ϴ,x)
    return jnp.mean((y_pred-y_true)**2)

In [ ]:
δxloss_at_x=jax.grad(loss_of_x)(x)
δxloss_at_x.shape

In [ ]:
G=jnp.mean(jnp.abs(δxloss_at_x),axis=0)
G.shape

Rappelons le noms de nos 19 features:

In [ ]:
feature_names=data_frame_clean.columns
feature_names

In [ ]:
fig,ax=plt.subplots()
x=range(19)
ax.bar(x=x,height=G)
ax.set_xticks(x)
ax.set_xticklabels(feature_names, rotation=90);

On voit que pour prédire la météo future du coin où l'on se trouve, il faut utiliser le barométre.

Attention: ces mesures d'importance sont à interpréter avec précaution: quand deux variable apportent des informations équivalentes, les gradient en sélectionne une au hasard.